In [73]:
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS

df = pd.read_excel("data5.xlsx", sheet_name=1, index_col='state_name', na_values='..')

# оставляем страны, по которым есть полные данные с 2003 по 2022
df = df.groupby('state_name').filter(lambda x: x['year'].min() == 2002 and x['year'].max() == 2022)

# удаляем чрезмерно экономически развитые страны (выбросы)
df = df.drop(['Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'Trinidad and Tobago', 
              'United Arab Emirates', 'Chile', 'Croatia', 'Hungary', 'Poland', 'Uruguay', 'Equatorial Guinea'])

# удаляем Океанию
df = df[df['region'] != 7]

In [74]:
# лагируем переменные
cols = ['libdem_norm_diff', 'property_rights_diff', 'tax_burden_diff', 'government_spending_diff', 
        'business_freedom_diff', 'monetary_freedom_diff', 'trade_freedom_diff', 'investment_freedom_diff', 
        'financial_freedom_diff', 'gov_effectiveness_diff', 'corruption_diff', 'pstab_diff']

for col in cols:
    df[f'{col}_lagged'] = df.groupby('state_name')[col].shift(-1)

In [75]:
# мультииндекс для регрессий (2022 исключён, т.к. для ряда нет лагированных зависимых переменных)
df = df[df['year'] != 2022].reset_index().set_index(['state_name', 'year'])

In [76]:
dependent_var = df['libdem_norm_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,libdem_norm_diff_lagged,R-squared:,0.0033
Estimator:,PanelOLS,R-squared (Between):,0.0634
No. Observations:,1740,R-squared (Within):,0.0010
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0033
Time:,22:10:33,Log-likelihood,-4879.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.1572
Entities:,87,P-value,0.3281
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1572


In [77]:
dependent_var = df['property_rights_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,property_rights_diff_lagged,R-squared:,0.0074
Estimator:,PanelOLS,R-squared (Between):,0.2279
No. Observations:,1740,R-squared (Within):,0.0030
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0074
Time:,22:10:33,Log-likelihood,-5547.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.5839
Entities:,87,P-value,0.0245
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.5839


In [78]:
dependent_var = df['tax_burden_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,tax_burden_diff_lagged,R-squared:,0.0252
Estimator:,PanelOLS,R-squared (Between):,0.3841
No. Observations:,1740,R-squared (Within):,0.0038
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0252
Time:,22:10:33,Log-likelihood,-4410.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,8.9733
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,8.9733


In [79]:
dependent_var = df['government_spending_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,government_spending_diff_lagged,R-squared:,0.0025
Estimator:,PanelOLS,R-squared (Between):,0.0445
No. Observations:,1740,R-squared (Within):,0.0020
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0025
Time:,22:10:33,Log-likelihood,-5548.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.8533
Entities:,87,P-value,0.5119
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.8533


In [80]:
dependent_var = df['business_freedom_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,business_freedom_diff_lagged,R-squared:,0.0015
Estimator:,PanelOLS,R-squared (Between):,0.0398
No. Observations:,1740,R-squared (Within):,0.0008
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0015
Time:,22:10:33,Log-likelihood,-5310.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.5209
Entities:,87,P-value,0.7607
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5209


In [81]:
dependent_var = df['monetary_freedom_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,monetary_freedom_diff_lagged,R-squared:,0.0015
Estimator:,PanelOLS,R-squared (Between):,0.0118
No. Observations:,1740,R-squared (Within):,0.0011
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0015
Time:,22:10:33,Log-likelihood,-5142.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.5126
Entities:,87,P-value,0.7669
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.5126


In [82]:
dependent_var = df['trade_freedom_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,trade_freedom_diff_lagged,R-squared:,0.0067
Estimator:,PanelOLS,R-squared (Between):,0.0605
No. Observations:,1740,R-squared (Within):,0.0058
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0067
Time:,22:10:33,Log-likelihood,-5569.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.3391
Entities:,87,P-value,0.0397
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,2.3391


In [83]:
dependent_var = df['investment_freedom_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,investment_freedom_diff_lagged,R-squared:,0.0036
Estimator:,PanelOLS,R-squared (Between):,-0.0464
No. Observations:,1740,R-squared (Within):,0.0050
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0036
Time:,22:10:33,Log-likelihood,-5575.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.2439
Entities:,87,P-value,0.2860
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.2439


In [84]:
dependent_var = df['financial_freedom_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,financial_freedom_diff_lagged,R-squared:,0.0009
Estimator:,PanelOLS,R-squared (Between):,-0.0314
No. Observations:,1740,R-squared (Within):,0.0015
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0009
Time:,22:10:33,Log-likelihood,-5354.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.3122
Entities:,87,P-value,0.9058
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.3122


In [85]:
dependent_var = df['gov_effectiveness_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,gov_effectiveness_diff_lagged,R-squared:,0.0018
Estimator:,PanelOLS,R-squared (Between):,0.0312
No. Observations:,1740,R-squared (Within):,0.0007
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0018
Time:,22:10:33,Log-likelihood,-5079.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.6337
Entities:,87,P-value,0.6740
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,0.6337


In [86]:
dependent_var = df['corruption_diff_lagged']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, entity_effects=False)
results = model.fit()

results.summary

Dep. Variable:,corruption_diff_lagged,R-squared:,0.0032
Estimator:,PanelOLS,R-squared (Between):,0.0441
No. Observations:,1740,R-squared (Within):,0.0015
Date:,"Wed, Mar 26 2025",R-squared (Overall):,0.0032
Time:,22:10:33,Log-likelihood,-4973.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.1310
Entities:,87,P-value,0.3417
Avg Obs:,20.000,Distribution:,"F(5,1735)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,1.1310


In [38]:
dependent_var = df['libdem_norm']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,libdem_norm,R-squared:,0.2861
Estimator:,PanelOLS,R-squared (Between):,0.7798
No. Observations:,1740,R-squared (Within):,-0.3415
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.7504
Time:,11:32:13,Log-likelihood,-7625.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,137.48
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,137.48


In [39]:
dependent_var = df['property_rights']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,property_rights,R-squared:,0.2169
Estimator:,PanelOLS,R-squared (Between):,0.5657
No. Observations:,1740,R-squared (Within):,0.0757
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.5401
Time:,11:32:41,Log-likelihood,-6828.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,95.019
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,95.019


In [40]:
dependent_var = df['tax_burden']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,tax_burden,R-squared:,0.2517
Estimator:,PanelOLS,R-squared (Between):,0.3584
No. Observations:,1740,R-squared (Within):,0.0165
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.3570
Time:,11:32:49,Log-likelihood,-6094.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,115.35
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,115.35


In [41]:
dependent_var = df['government_spending']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,government_spending,R-squared:,0.0991
Estimator:,PanelOLS,R-squared (Between):,-0.4912
No. Observations:,1740,R-squared (Within):,-0.0129
Date:,"Sat, Mar 29 2025",R-squared (Overall):,-0.4857
Time:,11:32:58,Log-likelihood,-7213.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,37.719
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,37.719


In [42]:
dependent_var = df['business_freedom']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,business_freedom,R-squared:,0.3348
Estimator:,PanelOLS,R-squared (Between):,0.5822
No. Observations:,1740,R-squared (Within):,0.0071
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.5737
Time:,11:33:15,Log-likelihood,-6557.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,172.62
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,172.62


In [43]:
dependent_var = df['monetary_freedom']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,monetary_freedom,R-squared:,0.0881
Estimator:,PanelOLS,R-squared (Between):,0.0226
No. Observations:,1740,R-squared (Within):,-0.0611
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.0219
Time:,11:33:26,Log-likelihood,-6567.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,33.158
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,33.158


In [44]:
dependent_var = df['trade_freedom']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,trade_freedom,R-squared:,0.2088
Estimator:,PanelOLS,R-squared (Between):,0.3933
No. Observations:,1740,R-squared (Within):,0.0932
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.3895
Time:,11:33:41,Log-likelihood,-6356.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,90.537
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,90.537


In [45]:
dependent_var = df['investment_freedom']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,investment_freedom,R-squared:,0.1858
Estimator:,PanelOLS,R-squared (Between):,0.3922
No. Observations:,1740,R-squared (Within):,-0.0954
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.3744
Time:,11:33:56,Log-likelihood,-7366.3
Cov. Estimator:,Unadjusted,,
,,F-statistic:,78.247
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,78.247


In [46]:
dependent_var = df['financial_freedom']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,financial_freedom,R-squared:,0.1743
Estimator:,PanelOLS,R-squared (Between):,0.5016
No. Observations:,1740,R-squared (Within):,-0.2022
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.4867
Time:,11:34:08,Log-likelihood,-7106.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,72.427
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,72.427


In [47]:
dependent_var = df['gov_effectiveness']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,gov_effectiveness,R-squared:,0.4432
Estimator:,PanelOLS,R-squared (Between):,0.8823
No. Observations:,1740,R-squared (Within):,-0.0813
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.8573
Time:,11:34:32,Log-likelihood,-7040.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,273.04
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,273.04


In [48]:
dependent_var = df['corruption']
independent_vars = df[['imf_pr', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,corruption,R-squared:,0.2542
Estimator:,PanelOLS,R-squared (Between):,0.6357
No. Observations:,1740,R-squared (Within):,-0.1693
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.6108
Time:,11:34:47,Log-likelihood,-7344.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,116.93
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(5,1715)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,116.93


In [49]:
dependent_var = df['imf_pr']
independent_vars = df[['libdem_norm', 'property_rights', 'tax_burden', 'government_spending', 
        'business_freedom', 'monetary_freedom', 'trade_freedom', 'investment_freedom', 
        'financial_freedom', 'gov_effectiveness', 'corruption', 'gdp_per_capita', 'fdi_per_capita', 'hdi_norm', 'pstab']]

model = PanelOLS(dependent_var, independent_vars, time_effects=True)
results = model.fit()

results.summary

Dep. Variable:,imf_pr,R-squared:,0.1786
Estimator:,PanelOLS,R-squared (Between):,0.5344
No. Observations:,1740,R-squared (Within):,0.0534
Date:,"Sat, Mar 29 2025",R-squared (Overall):,0.3575
Time:,11:35:33,Log-likelihood,-1003.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,24.708
Entities:,87,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(15,1705)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,24.708
